<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Reservoir%20Simulation%20Ertekin/Unit%205%20Finite-Difference%20Approximation%20to%20Linear-Flow%20Equations/fd1d_linearflow_explicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 5 Finite-Difference Approximations to Linear-Flow Problems**

## Explicit Formulation (*Forward-Difference Approximation*)

![Fig 5 29 Explicit Method](https://user-images.githubusercontent.com/51282928/76113357-94c97880-6016-11ea-8a3f-7658d9208cf9.PNG)

$$p_{i}^{n+1} = p^i_n + (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot q_{lsc_i} + (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot [T_{lx_{i+1/2}}^{n} p_{i+1}^{n}-(T_{lx_{i+1/2}}^{n} + T_{lx_{i-1/2}}^{n})p_{i}^{n}+T_{lx_{i-1/2}}p_{i-1}^{n}]$$

In the following codes, $(\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i$ will be written as `factor`

Transmissibility of coupling cells, $T_{lx_{i\pm1/2}}^{n}$, written as `T_min_half` or `T_plus_half`

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

fatal: destination path 'reservoir-engineering' already exists and is not an empty directory.


## Example 5.8 Five reservoir gridblocks with no flow at both boundaries

![Example 5 8](https://user-images.githubusercontent.com/51282928/75949840-09e36380-5eda-11ea-824c-dd5c9ac6f6d5.PNG)

### For timestep 10 days

In [56]:
"Task. determine pressure distribution during first year of production with timestep 10 days"

# known
p_initial = 6000 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 1000
delta_z = 75
ngrid = 5
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 10 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = 15 * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (alpha * B * delta_t) / (Vb * poro * c)
factor

1.1883597883597885

Because of uniform gridblocks, the equation for `tr_plus_coupling` and `tr_min_coupling` become:

$$T_{lx_{i+1/2}}^{n} = T_{lx_{i-1/2}}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i+1/2}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i-1/2}^{n}$$

In [57]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_y * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.1267875, 0.1267875)

There are 5 grids (`grid_1`, `grid_2`, `grid_3`, `grid_4`, `grid_5`), so the transmissibility coefficients will be: $$T_{lx_{i+1/2}}^{n}=(T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n}, T_{lx_{5+1/2}}^{n})$$

and

$$T_{lx_{i-1/2}}^{n}=(T_{lx_{1/2}}^{n}, T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n})$$

The values of $T_{lx_{5+1/2}}^{n}=0$ and $T_{lx_{1/2}}^{n}=0$ (the edge of gridblocks), so the values of each are:

$$T_{lx_{i+1/2}}^{n}=(0.127, 0.127, 0.127, 0.127, 0)$$

and

$$T_{lx_{i-1/2}}^{n}=(0, 0.127, 0.127, 0.127, 0.127)$$




In [58]:
q = np.full(ngrid-1, T_min_half)
Ti_plus_halves = np.append(q, [0])
print(Ti_plus_halves)
p = np.full(ngrid-1, T_plus_half)
Ti_min_halves = np.append([0], p)
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.1267875 0.1267875 0.1267875 0.1267875 0.       ]
[0.        0.1267875 0.1267875 0.1267875 0.1267875]
At grid 1, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.1267875 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.1267875 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.0 for T_plus_half.


Requires array for $q_{sc}$

In [0]:
qsc = [0, 0, 0, -150, 0] # production well in grid 4

Calculate $p_{i}^{n+1}$ for each grid in each time

In [60]:
pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

pi_arr = []
min_arr = []
plus_arr = []

for j in range(len(time)):
  pnew_arr = []

  minus = pi[0]
  plus = pi[-1]
  minus_arr = []
  plusus_arr = []
  for i, obj in enumerate(pi):
    if i > 0:
      minus = pi[i-1]
    if i < (len(pi) - 1):
      plus = pi[i+1]

    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
    pnew_arr.append(float(pnew))

    minus_arr.append(float(minus))
    plusus_arr.append(float(plus))

  pi = pnew_arr

  min_arr.append(minus_arr)
  plus_arr.append(plusus_arr)
  pi_arr.append(pi)

df = pd.DataFrame.from_records(pi_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5
0,15.0,6000.000000,6000.000000,6000.000000,5821.746032,6000.000000
1,25.0,6000.000000,6000.000000,5973.142623,5697.206817,5973.142623
2,35.0,6000.000000,5995.953421,5935.614184,5602.102885,5931.567605
3,45.0,5999.390305,5987.471853,5894.455577,5523.738961,5881.927430
4,55.0,5997.594562,5975.252912,5852.614698,5455.308514,5827.959472
5,65.0,5994.228364,5960.141312,5811.230704,5393.063347,5771.812463
6,75.0,5989.092497,5942.840943,5770.662014,5334.880120,5714.746649
7,85.0,5982.123813,5923.867570,5730.945175,5279.519220,5657.512476
8,95.0,5973.346394,5903.577533,5691.996659,5226.233153,5600.560547
9,105.0,5962.834378,5882.210835,5653.699174,5174.554980,5544.160950


### For timestep 15 days

In [61]:
delta_t = 15 # days

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (alpha * B * delta_t) / (Vb * poro * c)

pi = np.full(ngrid, 6000) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

pi_arr = []
min_arr = []
plus_arr = []

for j in range(len(time)):
  pnew_arr = []

  minus = pi[0]
  plus = pi[-1]
  minus_arr = []
  plusus_arr = []
  for i, obj in enumerate(pi):
    if i > 0:
      minus = pi[i-1]
    if i < (len(pi) - 1):
      plus = pi[i+1]

    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
    pnew_arr.append(float(pnew))

    minus_arr.append(float(minus))
    plusus_arr.append(float(plus))

  pi = pnew_arr

  min_arr.append(minus_arr)
  plus_arr.append(plusus_arr)
  pi_arr.append(pi)

df = pd.DataFrame.from_records(pi_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5
0,15.0,6000.000000,6000.000000,6000.000000,5732.619048,6000.000000
1,30.0,6000.000000,6000.000000,5939.570902,5586.096291,5939.570902
2,45.0,6000.000000,5986.342797,5873.341517,5478.488514,5859.684314
3,60.0,5996.913421,5963.890663,5809.641971,5386.497501,5773.532634
4,75.0,5989.450154,5936.493147,5748.870517,5302.220177,5686.061243
5,90.0,5977.481672,5906.058211,5690.329283,5222.533397,5599.311723
6,105.0,5961.339702,5873.444635,5633.361205,5146.029383,5514.158408
7,120.0,5941.475087,5839.049494,5577.482141,5071.985790,5430.959868
8,135.0,5918.326519,5803.082860,5522.353273,4999.978397,5349.830380
9,150.0,5892.281020,5765.682420,5467.740531,4929.723985,5270.762520


## Example 5.9 Similar to Ex 5.8, but **constant pressure at left boundary of reservoir**

![Example 5 9](https://user-images.githubusercontent.com/51282928/76109912-081bbc00-6010-11ea-9f00-023e9d954ac6.PNG)

The only modification is in the line:

```
pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
pnew_arr.append(float(pnew))
pnew_arr[0] = pi[0]
```
Where `pnew_arr[0] = pi[0]` means the pressure in the first gridblock is kept constant, equals to initial pressure `pi` 6000 psi.


### For timestep 15 days

In [62]:
pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

pi_arr = []
min_arr = []
plus_arr = []

for j in range(len(time)):
  pnew_arr = []

  plus = pi[-1]
  minus_arr = []
  plusus_arr = []
  for i, obj in enumerate(pi):
    if i > 0:
      minus = pi[i-1]
    if i < (len(pi) - 1):
      plus = pi[i+1]

    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
    pnew_arr.append(float(pnew))
    pnew_arr[0] = pi[0]

    minus_arr.append(float(minus))
    plusus_arr.append(float(plus))

  pi = pnew_arr

  min_arr.append(minus_arr)
  plus_arr.append(plusus_arr)
  pi_arr.append(pi)

df = pd.DataFrame.from_records(pi_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5
0,15.0,6000.0,6000.000000,6000.000000,5732.619048,6000.000000
1,30.0,6000.0,6000.000000,5939.570902,5586.096291,5939.570902
2,45.0,6000.0,5986.342797,5873.341517,5478.488514,5859.684314
3,60.0,6000.0,5963.890663,5809.641971,5386.497501,5773.532634
4,75.0,6000.0,5937.190726,5748.870517,5302.220177,5686.061243
5,90.0,6000.0,5908.824784,5690.486939,5222.533397,5599.311723
6,105.0,6000.0,5880.085553,5634.072855,5146.065014,5514.158408
7,120.0,6000.0,5851.586875,5579.381045,5072.166151,5430.967921
8,135.0,6000.0,5823.609260,5526.268116,5000.508212,5349.877375
9,150.0,6000.0,5796.274015,5474.644619,4930.909711,5270.918634


## Example 5.10 Halving the grid spacing of Ex 5.9 (more refined grids)

![Example 5 10](https://user-images.githubusercontent.com/51282928/76109917-0c47d980-6010-11ea-95bf-87492c658368.PNG)

### For timestep 15 days

In [63]:
# known
p_initial = 6000 # initial pressure, in psia
delta_x = 500 # ft, half the previous one
delta_y = 1000
delta_z = 75
ngrid = 10 # now twice the number of previous gridblocks
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 15 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = 15 * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (alpha * B * delta_t) / (Vb * poro * c)
factor

3.5650793650793653

In [64]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_y * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.253575, 0.253575)

In [67]:
q = np.full(ngrid-1, T_min_half)
Ti_plus_halves = np.append(q, [0])
print(Ti_plus_halves)
p = np.full(ngrid-1, T_plus_half)
Ti_min_halves = np.append([0], p)
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.253575 0.253575 0.253575 0.253575 0.253575 0.253575 0.253575 0.253575
 0.253575 0.      ]
[0.       0.253575 0.253575 0.253575 0.253575 0.253575 0.253575 0.253575
 0.253575 0.253575]
At grid 1, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.253575 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.253575 for T_min_half and: 0.253575 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.253575 for T_min_half and: 0.253575 for T_plus_half.


In [0]:
qsc = [0, 0, 0, 0, 0, 0, -75, -75, 0, 0] # production well in grid 7 and 8

In [74]:
pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

pi_arr = []
min_arr = []
plus_arr = []

for j in range(len(time)):
  pnew_arr = []

  plus = pi[-1]
  minus_arr = []
  plusus_arr = []
  for i, obj in enumerate(pi):
    if i > 0:
      minus = pi[i-1]
    if i < (len(pi) - 1):
      plus = pi[i+1]

    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
    pnew_arr.append(float(pnew))
    pnew_arr[0] = pi[0]

    minus_arr.append(float(minus))
    plusus_arr.append(float(plus))

  pi = pnew_arr

  min_arr.append(minus_arr)
  plus_arr.append(plusus_arr)
  pi_arr.append(pi)

df = pd.DataFrame.from_records(pi_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5', 'grid 6', 'grid 7', 'grid 8', 'grid 9', 'grid 10'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5,grid 6,grid 7,grid 8,grid 9,grid 10
0,15.0,6000.0,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,5.732619e+03,5.732619e+03,6.000000e+03,6.000000e+03
1,30.0,6000.0,6.000000e+03,6.000000e+03,6.000000e+03,6.000000e+03,5.758284e+03,5.706954e+03,5.706954e+03,5.758284e+03,6.000000e+03
2,45.0,6000.0,6.000000e+03,6.000000e+03,6.000000e+03,5.781485e+03,5.930397e+03,5.485976e+03,5.485976e+03,5.930397e+03,5.781485e+03
3,60.0,6000.0,6.000000e+03,6.000000e+03,5.802459e+03,6.113644e+03,5.394015e+03,5.620358e+03,5.620358e+03,5.394015e+03,5.916103e+03
4,75.0,6000.0,6.000000e+03,5.821420e+03,6.262355e+03,5.181772e+03,6.249188e+03,5.148360e+03,5.148360e+03,6.070608e+03,5.444128e+03
5,90.0,6000.0,5.838561e+03,6.381471e+03,4.886880e+03,7.123595e+03,4.289063e+03,5.876144e+03,5.714705e+03,4.670534e+03,6.010475e+03
6,105.0,6000.0,6.475303e+03,4.539539e+03,8.260037e+03,2.539111e+03,8.286268e+03,4.028075e+03,4.649321e+03,6.825807e+03,4.799148e+03
7,120.0,6000.0,4.295662e+03,9.652885e+03,-2.751512e+02,1.290643e+04,-7.587183e+02,8.171780e+03,5.787900e+03,3.026101e+03,6.631278e+03
8,135.0,6000.0,1.067942e+04,-4.165218e+03,2.061629e+04,-1.136342e+04,1.966809e+04,-2.323969e+03,5.178875e+03,8.781943e+03,3.372144e+03
9,150.0,6000.0,-6.970620e+03,3.165741e+04,-3.069670e+04,4.559966e+04,-2.826600e+04,2.407248e+04,1.386038e+03,6.341766e+02,8.262683e+03
